In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, find
import pickle

In [2]:
poorly_formatted_rows = [2120259, 2446317, 11141080, 11152098, 11152401, 11882086, 12902538, 12935043, 17589538]
df = pd.read_csv('lastfm_data/userid-timestamp-artid-artname-traid-traname.tsv', header=None, skiprows=poorly_formatted_rows, sep='\t')
df.columns = ['user_id', 'timestamp', 'artist_id', 'artist_name', 'track_id', 'track_name']
del df['timestamp'] #don't need timestamps.
df.dropna(subset=[['user_id','track_id']], inplace=True)

In [3]:
users = df.user_id.dropna().unique()
songs = df.track_id.dropna().unique()

In [4]:
utility_matrix = pd.DataFrame(index=songs, columns=users)
utility_matrix

,user_000001,user_000002,user_000003,user_000004,user_000005,user_000006,user_000007,user_000008,user_000009,user_000010,...,user_000991,user_000992,user_000993,user_000994,user_000995,user_000996,user_000997,user_000998,user_000999,user_001000
f7c1f8f8-b935-45ed-8fc8-7def69d92a10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
475d4e50-cebb-4cd0-8cd4-c3df97987962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dc394163-2b78-4b56-94e4-658597a29ef8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
340d9a0b-9a43-4098-b116-9f79811bd508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0b04407b-f517-4e00-9e6a-494795efc73e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4e78efc4-e545-47af-9617-05ff816d86e2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fb51d2c4-cc69-4128-92f5-77ec38d66859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4277434f-e3c2-41ae-9ce3-23fd157f9347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1151b040-8022-4965-96d2-8a4605ce456c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
f78c95a8-9256-4757-9a9f-213df5c6854e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
utility_matrix = utility_matrix.fillna(0)

In [6]:
utility_matrix

,user_000001,user_000002,user_000003,user_000004,user_000005,user_000006,user_000007,user_000008,user_000009,user_000010,...,user_000991,user_000992,user_000993,user_000994,user_000995,user_000996,user_000997,user_000998,user_000999,user_001000
f7c1f8f8-b935-45ed-8fc8-7def69d92a10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
475d4e50-cebb-4cd0-8cd4-c3df97987962,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dc394163-2b78-4b56-94e4-658597a29ef8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
340d9a0b-9a43-4098-b116-9f79811bd508,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0b04407b-f517-4e00-9e6a-494795efc73e,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4e78efc4-e545-47af-9617-05ff816d86e2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fb51d2c4-cc69-4128-92f5-77ec38d66859,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4277434f-e3c2-41ae-9ce3-23fd157f9347,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1151b040-8022-4965-96d2-8a4605ce456c,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
f78c95a8-9256-4757-9a9f-213df5c6854e,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
df_len = len(df)
for index, row in df.iterrows():
    prev_value = utility_matrix.loc[row['track_id']][row['user_id']]
    utility_matrix.loc[row['track_id']][row['user_id']] = prev_value + 1
    if (index % 10000 == 0):
        print("%d / %d : %.03f%%" %(index, df_len, (index/df_len)*100))


10000 / 16936134 : 0.059%
20000 / 16936134 : 0.118%
30000 / 16936134 : 0.177%
40000 / 16936134 : 0.236%
50000 / 16936134 : 0.295%
60000 / 16936134 : 0.354%
70000 / 16936134 : 0.413%
80000 / 16936134 : 0.472%
90000 / 16936134 : 0.531%
110000 / 16936134 : 0.649%
120000 / 16936134 : 0.709%
130000 / 16936134 : 0.768%
150000 / 16936134 : 0.886%
170000 / 16936134 : 1.004%
180000 / 16936134 : 1.063%
190000 / 16936134 : 1.122%
200000 / 16936134 : 1.181%
210000 / 16936134 : 1.240%
220000 / 16936134 : 1.299%
230000 / 16936134 : 1.358%
240000 / 16936134 : 1.417%
250000 / 16936134 : 1.476%
260000 / 16936134 : 1.535%
270000 / 16936134 : 1.594%
280000 / 16936134 : 1.653%
290000 / 16936134 : 1.712%
300000 / 16936134 : 1.771%
310000 / 16936134 : 1.830%
320000 / 16936134 : 1.889%
330000 / 16936134 : 1.948%
340000 / 16936134 : 2.008%
350000 / 16936134 : 2.067%
360000 / 16936134 : 2.126%
370000 / 16936134 : 2.185%
380000 / 16936134 : 2.244%
390000 / 16936134 : 2.303%
400000 / 16936134 : 2.362%
410000 / 1

3210000 / 16936134 : 18.954%
3220000 / 16936134 : 19.013%
3230000 / 16936134 : 19.072%
3240000 / 16936134 : 19.131%
3250000 / 16936134 : 19.190%
3260000 / 16936134 : 19.249%
3270000 / 16936134 : 19.308%
3280000 / 16936134 : 19.367%
3290000 / 16936134 : 19.426%
3300000 / 16936134 : 19.485%
3310000 / 16936134 : 19.544%
3320000 / 16936134 : 19.603%
3330000 / 16936134 : 19.662%
3340000 / 16936134 : 19.721%
3350000 / 16936134 : 19.780%
3360000 / 16936134 : 19.839%
3380000 / 16936134 : 19.957%
3390000 / 16936134 : 20.016%
3410000 / 16936134 : 20.134%
3420000 / 16936134 : 20.194%
3430000 / 16936134 : 20.253%
3440000 / 16936134 : 20.312%
3460000 / 16936134 : 20.430%
3480000 / 16936134 : 20.548%
3490000 / 16936134 : 20.607%
3500000 / 16936134 : 20.666%
3520000 / 16936134 : 20.784%
3530000 / 16936134 : 20.843%
3540000 / 16936134 : 20.902%
3560000 / 16936134 : 21.020%
3570000 / 16936134 : 21.079%
3580000 / 16936134 : 21.138%
3600000 / 16936134 : 21.256%
3610000 / 16936134 : 21.315%
3620000 / 1693

6440000 / 16936134 : 38.025%
6470000 / 16936134 : 38.202%
6480000 / 16936134 : 38.261%
6490000 / 16936134 : 38.320%
6500000 / 16936134 : 38.379%
6510000 / 16936134 : 38.439%
6520000 / 16936134 : 38.498%
6530000 / 16936134 : 38.557%
6540000 / 16936134 : 38.616%
6550000 / 16936134 : 38.675%
6560000 / 16936134 : 38.734%
6570000 / 16936134 : 38.793%
6580000 / 16936134 : 38.852%
6590000 / 16936134 : 38.911%
6600000 / 16936134 : 38.970%
6610000 / 16936134 : 39.029%
6620000 / 16936134 : 39.088%
6630000 / 16936134 : 39.147%
6640000 / 16936134 : 39.206%
6650000 / 16936134 : 39.265%
6660000 / 16936134 : 39.324%
6670000 / 16936134 : 39.383%
6680000 / 16936134 : 39.442%
6690000 / 16936134 : 39.501%
6700000 / 16936134 : 39.560%
6710000 / 16936134 : 39.619%
6720000 / 16936134 : 39.678%
6730000 / 16936134 : 39.738%
6740000 / 16936134 : 39.797%
6750000 / 16936134 : 39.856%
6760000 / 16936134 : 39.915%
6770000 / 16936134 : 39.974%
6780000 / 16936134 : 40.033%
6790000 / 16936134 : 40.092%
6800000 / 1693

9650000 / 16936134 : 56.979%
9660000 / 16936134 : 57.038%
9670000 / 16936134 : 57.097%
9680000 / 16936134 : 57.156%
9710000 / 16936134 : 57.333%
9720000 / 16936134 : 57.392%
9730000 / 16936134 : 57.451%
9740000 / 16936134 : 57.510%
9750000 / 16936134 : 57.569%
9760000 / 16936134 : 57.628%
9770000 / 16936134 : 57.687%
9790000 / 16936134 : 57.805%
9800000 / 16936134 : 57.864%
9810000 / 16936134 : 57.923%
9820000 / 16936134 : 57.983%
9830000 / 16936134 : 58.042%
9840000 / 16936134 : 58.101%
9850000 / 16936134 : 58.160%
9860000 / 16936134 : 58.219%
9870000 / 16936134 : 58.278%
9880000 / 16936134 : 58.337%
9890000 / 16936134 : 58.396%
9900000 / 16936134 : 58.455%
9910000 / 16936134 : 58.514%
9920000 / 16936134 : 58.573%
9930000 / 16936134 : 58.632%
9940000 / 16936134 : 58.691%
9950000 / 16936134 : 58.750%
9960000 / 16936134 : 58.809%
9970000 / 16936134 : 58.868%
9990000 / 16936134 : 58.986%
10000000 / 16936134 : 59.045%
10010000 / 16936134 : 59.104%
10030000 / 16936134 : 59.222%
10040000 / 

12760000 / 16936134 : 75.342%
12770000 / 16936134 : 75.401%
12780000 / 16936134 : 75.460%
12790000 / 16936134 : 75.519%
12810000 / 16936134 : 75.637%
12820000 / 16936134 : 75.696%
12840000 / 16936134 : 75.814%
12850000 / 16936134 : 75.873%
12860000 / 16936134 : 75.932%
12870000 / 16936134 : 75.991%
12880000 / 16936134 : 76.050%
12890000 / 16936134 : 76.109%
12900000 / 16936134 : 76.169%
12910000 / 16936134 : 76.228%
12920000 / 16936134 : 76.287%
12930000 / 16936134 : 76.346%
12940000 / 16936134 : 76.405%
12950000 / 16936134 : 76.464%
12960000 / 16936134 : 76.523%
12970000 / 16936134 : 76.582%
12980000 / 16936134 : 76.641%
12990000 / 16936134 : 76.700%
13010000 / 16936134 : 76.818%
13020000 / 16936134 : 76.877%
13030000 / 16936134 : 76.936%
13040000 / 16936134 : 76.995%
13050000 / 16936134 : 77.054%
13060000 / 16936134 : 77.113%
13070000 / 16936134 : 77.172%
13080000 / 16936134 : 77.231%
13090000 / 16936134 : 77.290%
13100000 / 16936134 : 77.349%
13110000 / 16936134 : 77.408%
13120000 /

15930000 / 16936134 : 94.059%
15940000 / 16936134 : 94.118%
15950000 / 16936134 : 94.177%
15960000 / 16936134 : 94.236%
15970000 / 16936134 : 94.295%
15980000 / 16936134 : 94.354%
15990000 / 16936134 : 94.414%
16000000 / 16936134 : 94.473%
16010000 / 16936134 : 94.532%
16020000 / 16936134 : 94.591%
16030000 / 16936134 : 94.650%
16040000 / 16936134 : 94.709%
16050000 / 16936134 : 94.768%
16060000 / 16936134 : 94.827%
16070000 / 16936134 : 94.886%
16080000 / 16936134 : 94.945%
16090000 / 16936134 : 95.004%
16100000 / 16936134 : 95.063%
16110000 / 16936134 : 95.122%
16120000 / 16936134 : 95.181%
16130000 / 16936134 : 95.240%
16140000 / 16936134 : 95.299%
16150000 / 16936134 : 95.358%
16160000 / 16936134 : 95.417%
16170000 / 16936134 : 95.476%
16180000 / 16936134 : 95.535%
16190000 / 16936134 : 95.594%
16200000 / 16936134 : 95.653%
16210000 / 16936134 : 95.713%
16220000 / 16936134 : 95.772%
16230000 / 16936134 : 95.831%
16240000 / 16936134 : 95.890%
16250000 / 16936134 : 95.949%
16260000 /

18860000 / 16936134 : 111.360%
18870000 / 16936134 : 111.419%
18880000 / 16936134 : 111.478%
18890000 / 16936134 : 111.537%
18900000 / 16936134 : 111.596%
18910000 / 16936134 : 111.655%
18920000 / 16936134 : 111.714%
18930000 / 16936134 : 111.773%
18940000 / 16936134 : 111.832%
18950000 / 16936134 : 111.891%
18960000 / 16936134 : 111.950%
18970000 / 16936134 : 112.009%
18980000 / 16936134 : 112.068%
18990000 / 16936134 : 112.127%
19000000 / 16936134 : 112.186%
19010000 / 16936134 : 112.245%
19020000 / 16936134 : 112.304%
19030000 / 16936134 : 112.363%
19040000 / 16936134 : 112.422%
19050000 / 16936134 : 112.481%
19060000 / 16936134 : 112.540%
19070000 / 16936134 : 112.599%
19090000 / 16936134 : 112.718%


In [8]:
utility_matrix

,user_000001,user_000002,user_000003,user_000004,user_000005,user_000006,user_000007,user_000008,user_000009,user_000010,...,user_000991,user_000992,user_000993,user_000994,user_000995,user_000996,user_000997,user_000998,user_000999,user_001000
f7c1f8f8-b935-45ed-8fc8-7def69d92a10,27,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
475d4e50-cebb-4cd0-8cd4-c3df97987962,37,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dc394163-2b78-4b56-94e4-658597a29ef8,22,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
340d9a0b-9a43-4098-b116-9f79811bd508,29,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0b04407b-f517-4e00-9e6a-494795efc73e,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4e78efc4-e545-47af-9617-05ff816d86e2,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fb51d2c4-cc69-4128-92f5-77ec38d66859,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4277434f-e3c2-41ae-9ce3-23fd157f9347,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1151b040-8022-4965-96d2-8a4605ce456c,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
f78c95a8-9256-4757-9a9f-213df5c6854e,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
i,j,v = find(utility_matrix.as_matrix())
#returns : (I,J,V) : tuple of arrays
# I,J, and V contain the row indices, column indices, and values of the nonzero matrix entries.
print(i,j,v)

In [11]:
from sys import getsizeof
getsizeof(utility_matrix)/1000/1000/1000 #Gb

7.753010722

#### 7.6 gigs in memory is too fucking much

In [21]:
import scipy
sparse_utility_matrix_np = scipy.sparse.csr_matrix(utility_matrix.values)

0.056

In [29]:
pickle.dump(sparse_utility_matrix_np, open( "utility_matrix.pickle", "wb" ) )

In [24]:
getsizeof(sparse_utility_matrix_np) #bytes

56

## 56 BYTES IS WAY TOO SMALL OMFG

In [28]:
sparse_utility_matrix_np.todense().shape

(960402, 992)